In [24]:
# %load Base User.py
import gc
import time
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as DF

data = pd.read_csv('/data/data/final_data.csv')

In [25]:
data.head()

,uid,mid,star,timeStamp
0,1722994,1306505,1,2007.0
1,1405477,10574468,1,2013.0
2,15849871,4910186,1,2011.0
3,1068524,1304643,1,2006.0
4,1307041,1851857,1,2008.0


In [26]:
train=data['uid'].unique()

In [14]:
test = pd.read_table('test.txt',sep='\t')

In [15]:
test.head()

,uid,mid1,mid2,mid3
0,1572864,NaN,NaN,NaN
1,1769472,NaN,NaN,NaN
2,12058627,NaN,NaN,NaN
3,1441797,NaN,NaN,NaN
4,1179655,NaN,NaN,NaN


In [27]:
testuser = test['uid'].unique()

In [32]:
train

array([ 1722994,  1405477, 15849871, ..., 14409233, 10555382,  1450565])

In [31]:
testuser

array([ 1572864,  1769472, 12058627, ...,  1048572,  1179645,  1048574])

In [51]:
#取test和train的交集
cross_user = pd.Series(list(set(train).intersection(set(testuser))))

In [34]:
len(cross_user)

47012

In [ ]:
for i in range(len(cross_user)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i

In [6]:
# uid 48613  mid 72541
mid = data['mid'].unique()
uid = data['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(cross_user)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
data['mid'] = data['mid'].map(lambda x:collect_mid_dencoder[x])
data['uid'] = data['uid'].map(lambda x:collect_uid_dencoder[x])

In [55]:
cross_user_df = pd.DataFrame(cross_user,columns=['uid'])

In [46]:
cross_user_df.head()

,uid
0,1572864
1,12058627
2,1441797
3,1179655
4,1572872


In [56]:
collect_testid_encoder={}
collect_testid_decoder={}
userid=cross_user.unique()
for i in range(len(cross_user)):
    collect_testid_encoder[i] = userid[i]
    collect_testid_decoder[userid[i]] = i
cross_user_df['uid'] = cross_user_df['uid'].map(lambda x:collect_testid_decoder[x])

In [57]:
cross_user_df.head()

,uid
0,0
1,1
2,2
3,3
4,4


In [7]:
len(uid)

48608

In [8]:
len(mid)

72581

In [9]:
user_matrix = np.zeros((len(uid),len(mid)),dtype='float16')
item_matrix = np.zeros((len(mid),len(uid)),dtype='float16')
data.head(5)

,uid,mid,star,timeStamp
0,0,0,1,2007.0
1,1,1,1,2013.0
2,2,2,1,2011.0
3,3,3,1,2006.0
4,4,4,1,2008.0


In [10]:
DF(user_matrix,dtype=np.uint16).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48608 entries, 0 to 48607
Columns: 72581 entries, 0 to 72580
dtypes: uint16(72581)
memory usage: 6.6 GB


In [29]:
to_make = np.array(data[['uid','mid','star']].values,dtype=np.uint32)
for i in range(len(to_make)):
    if i%1000000 == 0:
        print(i)
    user_matrix[to_make[i][0],to_make[i][1]] = to_make[i][2]
    item_matrix[to_make[i][1],to_make[i][0]] = to_make[i][2]

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000


In [11]:
user_matrix.shape

(48608, 72581)

In [12]:
user_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [13]:
user_matrix.shape

(48608, 72581)

In [37]:
test = user_matrix.sum(axis=1)

In [38]:
test

array([1.400e+02, 4.590e+02, 1.501e+03, ..., 1.000e+00, 1.000e+00,
       1.000e+00], dtype=float16)

In [39]:
test.shape

(48608,)

In [41]:
test_matrix = (1+0.0)/test

In [43]:
min(test_matrix)

0.000292

In [44]:
max(test_matrix)

1.0

In [10]:
#1000-5000个做训练集，0-1000个做测试
split_test = user_matrix[:1000]
split_train = user_matrix[1000:5000]
print(split_train.shape)
print(split_test.shape)

(4000, 72656)
(1000, 72656)


In [11]:
# import random
# new_vector = np.zeros(len(mid)) # readline
# for i in range(200000):
#     kfc = random.randint(1,len(uid))
#     new_vector[kfc] = random.randint(0,5)
    
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according
    to the definition of the dot product
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    
def calc_sim(vec,user_matrix):
    ans = []
    for i in range(len(user_matrix)):
        if i % 10000 ==0:
            print(i)
        #ans.append(np.linalg.norm(vec-user_matrix[i]))
        ans.append(cos_sim(vec,user_matrix[i]))
    return ans

In [12]:
split_test

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       [0., 0., 5., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [ ]:
now = 0
final_acc = []
for new_vec in split_test:
    print(now)
    ans = calc_sim(new_vec,split_train)
    to_recommand = DF(ans).reset_index().rename(columns={0:'sim'})
    k = 3
    topk = list(to_recommand.sort_values(by=['sim'],ascending=False).head(k)['index'].values)

In [14]:
ans=calc_sim(split_test[0],split_train)

0


In [16]:
 to_recommand = DF(ans).reset_index().rename(columns={0:'sim'})

In [18]:
to_recommand.head()

,index,sim
0,0,0.125366
1,1,0.159302
2,2,0.247681
3,3,0.097534
4,4,0.022690


In [20]:
k=3
topk = list(to_recommand.sort_values(by=['sim'],ascending=False).head(k)['index'].values)

In [21]:
topk

[2550, 2464, 2044]

In [26]:
data.head()

,uid,mid,timeStamp,star
0,0,0,2007-08-22,5.0
1,1,1,2013-04-24,4.0
2,2,2,2011-12-20,5.0
3,3,3,2006-02-07,3.0
4,4,4,2008-09-22,4.0


In [23]:
for i in topk:
    watch_set = watch_set|set(data[data['uid']==i]['mid'].unique())

In [29]:
topN = []
for i in list(watch_set):
    temp = 0
    for j in topk:
        temp += to_recommand[to_recommand['index']==j]['sim'].values*user_matrix[j][i]
    topN.append(temp)

topN = DF(topN).rename(columns={0:'score'})

In [30]:
topN

,score
0,3.000000
1,5.352539
2,4.391602
3,8.430176
4,4.000000
5,1.739502
6,1.014404
7,1.739502
8,4.000000
9,6.744141


In [27]:
import random
now = 0
final_acc = []
for new_vec in user_matrix:
    print(now)
    ans = calc_sim(new_vec,user_matrix)
    to_recommand = DF(ans).reset_index().rename(columns={0:'sim'})
    k = 3
    #找出最相似的三个用户
    topk = list(to_recommand.sort_values(by=['sim'],ascending=False).head(k)['index'].values)
#     print(topk)
#     no_to_use = []
#     for i in range(len(new_vec)):
#         if random.randint(0,100) % 10 == 0:
#             if new_vec[i]!=0:
#                 no_to_use.append(i)
    
    # 推荐没给他看过的
    watch_set = set()
    #求三个用户的并集
    for i in topk:
        watch_set = watch_set|set(data[data['uid']==i]['mid'].unique())

    watch_set = set(watch_set)
    print('The New One No Watch Movie Length: ',len(watch_set))
    
    topN = []
    for i in list(watch_set):
        temp = 0
        for j in topk:
            temp += to_recommand[to_recommand['index']==j]['sim'].values*user_matrix[j][i]
        topN.append(temp)

    topN = DF(topN).rename(columns={0:'score'})
    topN['mid'] = watch_set
    need_to_show = 50
    show_video = topN.sort_values(by=['score'],ascending=False).head(need_to_show)['mid'].values
    true_video = [collect_mid_encoder[i] for i in show_video]
#     print(show_video)
#     print(true_video)
#     print(topN.sort_values(by=['score'],ascending=False).head(need_to_show))
    acc = data[(data['mid'].isin(show_video) & (data['uid']==now))].shape[0]/need_to_show
    print('True: ',acc)
    final_acc.append(acc)
    now += 1

0
0
10000
20000
30000
40000
The New One No Watch Movie Length:  352
True:  1.0
1
0
10000
20000
30000
40000
The New One No Watch Movie Length:  1320
True:  1.0
2
0
10000
20000
30000
40000
The New One No Watch Movie Length:  3099
True:  1.0
3
0
10000
20000
30000
40000
The New One No Watch Movie Length:  327
True:  1.0
4
0
10000
20000
30000
40000
The New One No Watch Movie Length:  1064
True:  1.0
5
0
10000
20000
30000
40000
The New One No Watch Movie Length:  949
True:  2.8
6
0
10000
20000
30000
40000
The New One No Watch Movie Length:  1175
True:  1.0
7
0
10000
20000
30000
40000
The New One No Watch Movie Length:  1576
True:  1.0
8
0
10000
20000
30000
40000
The New One No Watch Movie Length:  1785
True:  1.0
9
0
10000
20000
30000
40000
The New One No Watch Movie Length:  775
True:  109.0
10
0
10000
20000
30000
40000
The New One No Watch Movie Length:  3672
True:  1.0
11
0
10000
20000
30000
40000
The New One No Watch Movie Length:  1188
True:  2.6
12
0
10000
20000
30000
40000
The New One

KeyboardInterrupt: 

In [57]:
print(np.mean(final_acc))  # 766

0.7262000000000001


In [ ]:
data[data['uid']==0][['mid','star']]

In [56]:
for i in range(len(user_matrix[0])):
    if user_matrix[0][i]!=0:
        print(i)

0
53
79
80
93
113
126
141
145
152
162
172
176
192
196
225
266
268
278
281
289
307
313
347
354
357
358
365
370
410
430
454
461
500
512
515
537
562
563
596
603
615
616
620
622
629
630
649
654
674
687
688
723
738
753
758
794
818
838
849
864
928
949
961
980
1014
1019
1040
1119
1123
1128
1146
1150
1161
1302
1324
1363
1366
1371
1450
1480
1503
1536
1684
1845
1886
1896
1921
1981
2035
2044
2324
2365
2367
2391
2470
2637
2638
2685
2825
2917
2935
2966
3041
3136
3168
3237
3250
3279
3287
3376
3444
3509
3522
3544
3562
3590
3597
3642
3647
3663
3769
3783
3793
3828
3943
4072
4544
4576
4744
5209
5304
5549
6238
6433
6629
8270
8522
11900
16354


In [58]:
# data.sort_values(by=['uid','mid'])
data[data['uid']==0].nunique()

uid            1
mid          140
timeStamp     63
star           3
dtype: int64

1